In [1]:
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge, LassoCV, RidgeCV, Lasso, LogisticRegressionCV
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV

from sklearn.metrics import roc_auc_score, mean_squared_error, roc_curve, RocCurveDisplay, auc

from scipy import stats as st


from matplotlib import pyplot as plt

In [3]:
data = pd.read_csv('./data/movieReplicationSet.csv')
data.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1


In [4]:
movie_ratings = data.iloc[:,:400]

In [5]:
movie_ratings.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,X-Men 2 (2003),The Usual Suspects (1995),The Mask (1994),Jaws (1975),Harry Potter and the Chamber of Secrets (2002),Patton (1970),Anaconda (1997),Twister (1996),MacArthur (1977),Look Who's Talking (1989)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,0.5,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,NaN,3.0,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.5,NaN,3.0,NaN,NaN,NaN,NaN,1.5,NaN,NaN


# Question 1

In [6]:
# For data imputation:
avg_rating_of_user = np.mean(movie_ratings, axis = 1)
avg_rating_of_movie = np.mean(movie_ratings, axis = 0)

num_users = avg_rating_of_user.shape[0]
num_movies = avg_rating_of_movie.shape[0]

movie_avg = (np.ones((num_users,num_movies))*np.reshape(np.asarray(avg_rating_of_movie), (1,num_movies)))

user_avg = (np.ones((num_users,num_movies))*np.reshape(np.asarray(avg_rating_of_user), (num_users,1)))

imputation_replacement = (movie_avg + user_avg)/2
imputation_replacement.shape

(1097, 400)

In [7]:
invalid_vals = movie_ratings.isna()
invalid_vals.shape

(1097, 400)

In [8]:
for i in range(1097):
    for j in range(400):
        if invalid_vals.iloc[i][j]:
            movie_ratings.iloc[i][j] = imputation_replacement[i][j]

movie_ratings.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,X-Men 2 (2003),The Usual Suspects (1995),The Mask (1994),Jaws (1975),Harry Potter and the Chamber of Secrets (2002),Patton (1970),Anaconda (1997),Twister (1996),MacArthur (1977),Look Who's Talking (1989)
0,2.447086,2.381992,4.000000,2.725235,3.000000,2.670257,2.554121,2.821232,2.619604,2.827211,...,2.828460,2.921947,2.650951,4.000000,0.500000,2.510773,2.519156,2.572578,2.428806,2.540410
1,2.439294,2.374200,1.500000,2.717443,2.752945,2.662464,2.546329,2.813440,2.611812,2.819419,...,2.820668,2.914154,2.643159,2.673112,4.000000,2.502981,2.511364,2.564786,2.421013,2.532618
2,2.733065,2.667971,3.234118,3.011214,3.046716,2.956236,2.840100,3.107211,2.905583,3.113190,...,3.114439,3.207926,2.936930,2.966883,3.500000,2.796752,2.805135,2.858557,2.714784,2.826389
3,2.282975,2.217880,2.000000,2.561123,3.000000,2.506145,2.390009,2.657120,2.455492,4.000000,...,2.664348,3.000000,2.486840,2.516793,2.500000,2.346661,2.355044,2.408466,2.264694,2.376299
4,2.209132,2.144038,3.500000,2.487281,0.500000,2.432303,0.500000,1.000000,2.381650,0.000000,...,2.500000,2.683993,3.000000,2.442950,2.769704,2.272819,2.281202,1.500000,2.190852,2.302456


In [ ]:
# Linear Regression models per movie
movie_list = list(movie_ratings.keys())

scores = {}

for movie_to_predict in movie_list:
    
    scores[movie_to_predict] = {}
    
    target = movie_ratings[movie_to_predict]
        
    for movie_as_input_index in range(len(movie_list)):
        
        movie_as_input = movie_list[movie_as_input_index]
        
        if movie_as_input == movie_to_predict:
            continue
        
        inp = movie_ratings[movie_as_input]

        valid = ~target.isna()
        
        inp = np.reshape(np.asarray(inp[valid]), (-1,1))
        y_true = np.reshape(np.asarray(target[valid]), (-1,1))
                
        model = LinearRegression().fit(inp,y_true)
        r2 = model.score(inp,y_true)
        scores[movie_to_predict][movie_as_input] = r2
        print(movie_list.index(movie_to_predict), end = '\r')
#         print("\nFor predicting {} using {}, the R2 value is:\n{}\n\n\n".format(movie_to_predict, movie_as_input, r2))


In [ ]:
# best input-target movie pairs and avg of the 400

best_r2_scores = []

target_input_r2 = []

for target_movie in scores:
    predictor_movies = [i for i  in scores[target_movie]]
    r2_scores = [scores[target_movie][i] for i in predictor_movies]
    best_r2_index = np.argmax(r2_scores)
    target_input_r2.append([target_movie, predictor_movies[best_r2_index], r2_scores[best_r2_index]])
    print("For predicting {}, the best predictor is {} and the R2 is {}".format(target_movie, predictor_movies[best_r2_index], r2_scores[best_r2_index]))
    best_r2_scores.append(r2_scores[best_r2_index])
    

In [ ]:
print('Average COD of best LR models for each of the 400 movies is: ', np.mean(best_r2_scores))

In [ ]:
plt.hist(best_r2_scores, bins = 20)

In [ ]:
target_input_r2 = pd.DataFrame(target_input_r2)
target_input_r2.columns = ['Target','Predictor', 'R2 Value']
# target_input_r2
sorted_by_r2 = target_input_r2.iloc[np.argsort(target_input_r2['R2 Value'])]
easiest_to_predict = sorted_by_r2.iloc[-10:]
hardest_to_predict = sorted_by_r2.iloc[:10]

In [ ]:
easiest_to_predict

In [ ]:
hardest_to_predict

# Question 2

In [ ]:
data.iloc[:,474:477].isna().sum()

In [ ]:
gender, only, solo = data.keys()[474:477]
gender

In [ ]:
for i in hardest_to_predict.iloc:
    print(i)

In [ ]:
# Multiple regression models for hardest to predict movies:

gender, only, solo = data.keys()[474:477]

gender_mode = st.mode(data[gender])
data[gender][data[gender].isna()] = float(gender_mode.mode)

delta_r2 = []

for target_input_r2 in hardest_to_predict.iloc:
    
    target,best_predictor,old_r2 = target_input_r2
    
    input_data_subset = pd.DataFrame()
    input_data_subset['input_movie'] = movie_ratings[best_predictor]
    input_data_subset['gender_identity'] = data[gender]
    input_data_subset['only_child'] = data[only]
    input_data_subset['solo_watcher'] = data[solo]
    
#     print(input_data_subset.isna().sum())
    
    target_ratings = movie_ratings[target]
    
#     inp = np.reshape(np.asarray(inp[valid]), (-1,1))
#     y_true = np.reshape(np.asarray(target[valid]), (-1,1))
    
    valid = ~target_ratings.isna()
    model = LinearRegression().fit(input_data_subset[valid],target_ratings[valid])
    new_r2 = model.score(input_data_subset[valid],target_ratings[valid])
    
    print("R2 value for {} has changed from {} to {}\n\n".format(target,old_r2,new_r2))
    
    delta_r2.append([target,best_predictor, old_r2, new_r2])
    

In [ ]:
# Multiple regression models for easiest to predict movies:

# delta_r2 = []
# delta_r2 will have the first 10 entries for hardest to predict and next 10 for easiest to predict

for target_input_r2 in easiest_to_predict.iloc:
    
    target,best_predictor,old_r2 = target_input_r2
    
    input_data_subset = pd.DataFrame()
    input_data_subset['input_movie'] = movie_ratings[best_predictor]
    input_data_subset['gender_identity'] = data[gender]
    input_data_subset['only_child'] = data[only]
    input_data_subset['solo_watcher'] = data[solo]
    
#     print(input_data_subset.isna().sum())
    
    target_ratings = movie_ratings[target]
    
#     inp = np.reshape(np.asarray(inp[valid]), (-1,1))
#     y_true = np.reshape(np.asarray(target[valid]), (-1,1))
    
    valid = ~target_ratings.isna()
    model = LinearRegression().fit(input_data_subset[valid],target_ratings[valid])
    new_r2 = model.score(input_data_subset[valid],target_ratings[valid])
    
    print("R2 value for {} has changed from {} to {}\n\n".format(target,old_r2,new_r2))
    
    delta_r2.append([target,best_predictor, old_r2, new_r2])
    

In [ ]:
delta_r2

In [ ]:
delta_r2 = np.asarray(delta_r2)
x = [float(i) for i in delta_r2[:,2]]
y = [float(i) for i in delta_r2[:,3]]
plt.scatter(x,y)
plt.xlabel('R2 Values with just 1 movie')
plt.ylabel('New R2 values with extra attributes')

In [ ]:
diff = [y[i] - x[i] for i in range(20)]
plt.plot(diff)
plt.title('Difference between old and new R2 values')

# Question 3

In [ ]:
moderately_predictable = sorted_by_r2.iloc[184:215]
moderately_predictable

In [ ]:
moderately_predictable_movies = np.asarray(moderately_predictable['Target'])
input_candidates = []
for i in movie_list:
    if i not in moderately_predictable_movies and i not in delta_r2[:,0]:
        input_candidates.append(i)
        
input_movies = np.random.choice(input_candidates, 10)
input_movies

In [ ]:
movie_ratings.iloc[896] = np.mean(movie_ratings,axis = 0)

In [ ]:
input_data_subset = movie_ratings[input_movies]

optimal_alphas = []
best_rmse_coefs = []
rmse_scores =[]

for i in range(30):
    target_ratings = movie_ratings[moderately_predictable_movies[i]]

    X_train, X_test, y_train, y_test = train_test_split(input_data_subset, target_ratings, test_size = 0.2, random_state = 42)

    model = RidgeCV(alphas = np.arange(1,400,1), cv = 10, scoring = 'neg_root_mean_squared_error').fit(X_train,y_train)
    test_rmse = mean_squared_error(model.predict(X_test), y_test,squared= False)
    rmse_scores.append(test_rmse)

    optimal_alphas.append(model.alpha_)
    
    best_rmse_coefs.append(model.coef_)


In [ ]:
for i in range(30):
    print("\nFor movie: ", moderately_predictable_movies[i],"\tRMSE value is ", rmse_scores[i])

# Question 4

In [ ]:
# best_alpha_lasso = [] 
# RMSE_values_lasso = []
# lasso_weight = []
# for i in range(len(moderately_predictable_movies)):
#     x_mid = df_movies_copy[k2]
#     y_mid = df_movies_copy[moderately_predictable_movies[i]]
#     x_train,x_test,y_train,y_test=train_test_split(x_mid,y_mid,test_size=0.2,random_state =42)
#     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
#     parameters = {'alpha':[2.0**c for c in np.arange(-50, 0)]}
#     # define the model/ estimator
#     model = Lasso()
#     # define the grid search
#     Lasso_reg= GridSearchCV(model, parameters, scoring='neg_mean_squared_error',cv=cv)
#     #fit the grid search
#     Lasso_reg.fit(x_train,y_train)
#     best_model_lasso = Lasso_reg.best_estimator_
#     best_model_lasso.fit(x_train,y_train)
#     #model = LassoCV(alphas=np.arange(0, 1, 0.001), cv=cv)
#     #model.fit(x_train, y_train)
#     best_alpha_lasso.append(best_model_lasso.alpha)
#     RMSE = mean_squared_error(best_model_lasso.predict(x_test), y_test)
#     RMSE_values_lasso.append(RMSE)
#     lasso_weight.append(best_model_lasso.coef_)
#     print('movie done')

In [ ]:
input_data_subset = movie_ratings[input_movies]

best_alpha_lasso = [] 
RMSE_values_lasso = []
lasso_weight = []

for i in range(30):
    target_ratings = movie_ratings[moderately_predictable_movies[i]]

    x_train, x_test, y_train, y_test = train_test_split(input_data_subset, target_ratings, test_size = 0.2, random_state = 42)

    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
    parameters = {'alpha':[2.0**c for c in np.arange(-50, 0)]}
    
    model = Lasso()
    
    Lasso_reg= GridSearchCV(model, parameters, scoring='neg_mean_squared_error',cv=cv)
    
    Lasso_reg.fit(x_train,y_train)
    best_model_lasso = Lasso_reg.best_estimator_
    best_model_lasso.fit(x_train,y_train)
    
    best_alpha_lasso.append(best_model_lasso.alpha)
    RMSE = mean_squared_error(best_model_lasso.predict(x_test), y_test)
    RMSE_values_lasso.append(RMSE)
    lasso_weight.append(best_model_lasso.coef_)

In [ ]:
for i in range(len(best_alpha_lasso)):
    print("For movie: ", moderately_predictable_movies[i])
    print("RMSE value is ", RMSE_values_lasso[i])
    print("Optimal Alpha value is ", best_alpha_lasso[i])
    print("Norm of the coefficients is ", np.linalg.norm(lasso_weight[i]),"\n\n\n")

In [ ]:
for i in range(len(best_alpha_lasso)):
    print("\nFor movie: ", moderately_predictable_movies[i],"\tRMSE value is ", RMSE_values_lasso[i])

## Question 5

In [ ]:
avg_rating_of_user[896] = np.mean(avg_rating_of_user)
sorted_order = np.argsort(avg_rating_of_movie)

target_movies = movie_ratings.keys()[sorted_order][198:202]

In [ ]:
r2_scores = []
auc_vals = []
for target_movie in target_movies:
#     print(target_movie)
    target_median = np.median(movie_ratings[target_movie])
    y_label = 1*(movie_ratings[target_movie] >= target_median)
    X_train, X_test, y_train, y_test = train_test_split(avg_rating_of_user, y_label, test_size = 0.2, random_state = 42)
#     print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    model = LogisticRegressionCV(cv = 10, random_state = 42).fit(np.reshape(np.asarray(X_train), (-1,1)), y_train)
    r2 = model.score(np.reshape(np.asarray(X_test), (-1,1)), y_test)
    r2_scores.append(r2)
    preds = model.predict(np.reshape(np.asarray(X_test), (-1,1)))
    auc_vals.append(roc_auc_score(y_test,preds))
    plt.figure()
    fpr1, tpr1, thresholds1 = roc_curve(y_test, preds)
    roc_auc1 = auc(fpr1, tpr1)
    display1 = RocCurveDisplay(fpr=fpr1, tpr=tpr1, roc_auc=roc_auc1, estimator_name='Fahrenheit 9/11 (2004)')
    display1.plot()
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC curve for {}".format(target_movie))
    

In [ ]:
for i in range(4):
    print("For predicting movie {}, the AUC is {}".format(target_movies[i],auc_vals[i]))

# Extra Credit

In [ ]:
# Multiple regression models for hardest to predict movies:

v1,v2,v3 = data.keys()[474:477]

gender_mode = st.mode(data[gender])
data[gender][data[gender].isna()] = float(gender_mode.mode)

delta_r2 = []

for target_input_r2 in hardest_to_predict.iloc:
    
    target,best_predictor,old_r2 = target_input_r2
    
    input_data_subset = pd.DataFrame()
    input_data_subset['input_movie'] = movie_ratings[best_predictor]
    input_data_subset['variable_1'] = data[v1]
    input_data_subset['variable_2'] = data[v2]
    input_data_subset['variable_3'] = data[v3]
    
#     print(input_data_subset.isna().sum())
    
    target_ratings = movie_ratings[target]
    
#     inp = np.reshape(np.asarray(inp[valid]), (-1,1))
#     y_true = np.reshape(np.asarray(target[valid]), (-1,1))
    
    valid = ~target_ratings.isna()
    model = LinearRegression().fit(input_data_subset[valid],target_ratings[valid])
    new_r2 = model.score(input_data_subset[valid],target_ratings[valid])
    
    print("R2 value for {} has changed from {} to {}\n\n".format(target,old_r2,new_r2))
    
    delta_r2.append([target,best_predictor, old_r2, new_r2])
    